In [1]:
### Make adjustments to data for MOFA input from E1 + Prepare

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
   

In [3]:
source('MS4_Plot_Config.r')

###############################################
# Preqrequisites Configurations & Parameters

In [4]:
data_path = '../data/current'

In [5]:
result_path = '../results/current'

In [6]:
data_path

[1] "../data/current"

In [7]:
standardize = FALSE

In [8]:
set_zero_na = FALSE

In [9]:
quantile_norm_feat = TRUE

In [10]:
samples_var = 'all'
# samples_var = 'acs'

In [11]:
name = 'V_AZIMUTH'   # name of dataset to load
name_save = 'V_AZIMUTH_INTEGRATED'

# Functions

In [12]:
### Function for quantile normalization

quantile_normalization = function(X){
  set.seed(42)
  ranks = apply(X, 2, rank, ties.method = 'min')  # determine ranks of each entry
  
  sorted = data.frame(apply(X, 2, sort)) # sort the entries
  means = apply(sorted, 1, mean) # calculate the means
  
  normalized_data = apply(ranks, 2 ,function(x){ means[x]}) # substitute the means into ranks matrix
}


In [13]:
### Gene wise quantile normalization

stdnorm <- function(x) {
  set.seed(42)
  r = rank(x[!is.na(x)], ties.method="average")
  x[!is.na(x)] = qnorm(r / (length(x[!is.na(x)]) + 1))
  return(x)
}

# Load Data 

## Prepared combined data

In [14]:
path = paste0(result_path, '/G-Analysis/Combined_Data_',name,'.csv')
data_long = read.csv(path)
print(file.info(path)$mtime)

[1] "2023-02-01 11:41:54 CET"


In [15]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,B.cell__PARK7,0.9412188,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,B.cell__ENO1,0.8205378,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [16]:
nrow(data_long)

[1] 1065636

In [17]:
nrow(unique(data_long[,c('sample_id', 'type', 'variable')]))

[1] 1065636

In [21]:
length(unique(data_long$sample_id))

[1] 146

## Sample Data

In [24]:
sample_data = read.csv(paste0(result_path, '/00_Data_Overview/Merged_Sample_Meta_Data.csv'))

In [26]:
sample_data$CK_raw = sample_data$CK

In [27]:
sample_data$CK = log2(sample_data$CK+1)

In [28]:
sample_data$Troponin = log2(sample_data$Troponin+1)

In [29]:
sample_data$CK_MB = log2(as.numeric(sample_data$CK_MB)+1)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [30]:
sample_data$CRP =  log2(as.numeric(sample_data$CRP)+1)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [31]:
sample_data$sample_id[(sample_data$sample_id == 'm13.2')  & (sample_data$library %in%  c('L6'))] = 'm13.22'

In [32]:
sample_data$sample_id[(sample_data$sample_id == 'm6.4')  & (sample_data$library %in%  c('L10'))] = 'm6.42'

In [33]:
sample_data$sample_id[(sample_data$sample_id == 'm6.1')  & (sample_data$library %in%  c('L3'))] = 'm6.12'

# Potential Pre-Processing / Data Adjustments

###  Filter samples (remove no-acs patients)

In [54]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,B.cell__PARK7,0.9412188,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,B.cell__ENO1,0.8205378,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [55]:
head(sample_data,2)

,X.1,sample_id,sample,id,measurement,library,id.y,name,read,pattern,⋯,delta_ef_value_group,delta_ef_value,delta_ef_value_class,ef_classification_data,CK,CK_MB,Troponin,CRP,clinical_data,CK_raw
,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,1,k1,K1,1,TP0,L13,HTO_B0251,No-CCS-1,R2,5PNNNNNNNNNN(BC),⋯,NA,NA,NA,0,5.459432,NA,0.01863417,0.4854268,1,43
2,2,k10,K10,10,TP0,L11,HTO_B0256,Ch-CCS-10,R2,5PNNNNNNNNNN(BC),⋯,NA,NA,NA,0,NA,NA,NA,0.2630344,1,NA


In [56]:
unique(sample_data$classification)

[1] "vollstaendiger_ausschluss" "ccs"                      
[3] "koronarsklerose"           "acs_subacute"             
[5] "acs_w_infection"           "acs_w_o_infection"

In [57]:
samples_var

[1] "all"

In [58]:
if(samples_var == 'all'){
    samples = unique(sample_data$sample_id) # without filter 
    }
if(samples_var == 'acs_ccs'){
    samples = unique(sample_data$sample_id[!sample_data$classification %in% c('vollstaendiger_ausschluss', 'koronarsklerose')])  # only acs comparison
    }
if(samples_var == 'no_acs'){
    samples = unique(sample_data$sample_id[sample_data$classification %in% c('vollstaendiger_ausschluss', 'ccs', 'koronarsklerose')])  # only acs comparison
    }
if(samples_var == 'acs'){
    samples = unique(sample_data$sample_id[sample_data$classification %in% c('acs_subacute', 'acs_w_infection', 'acs_w_o_infection')])  # only acs comparison
    }
    

In [59]:
length(samples)

[1] 149

In [60]:
#samples

In [61]:
length(unique(data_long$sample_id))

[1] 146

In [62]:
data_long = data_long[data_long$sample_id %in% samples,]

In [63]:
length(unique(data_long$sample_id))

[1] 146

### Filter out variables

In [64]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,B.cell__PARK7,0.9412188,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,B.cell__ENO1,0.8205378,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [65]:
unique(data_long$type)

[1] "single_cell"   "cytokine"      "proteomics"    "neutrophil"   
[5] "clinical_data"

In [66]:
data_long = data_long[data_long$variable != 'delta_ef_value',]

In [67]:
unique(data_long[data_long$type == 'clinical_data', c('type', 'variable')])

,type,variable
,<chr>,<chr>
1064907,clinical_data,CK
1065053,clinical_data,CK_MB
1065199,clinical_data,Troponin
1065491,clinical_data,CRP


### Filter on expression

In [68]:
data_long$expression = TRUE

In [69]:
data_long$expression[data_long$value == 0] = FALSE

In [71]:
expression_filter = data_long %>% group_by(type, variable) %>% summarise(perc_expression = sum(expression)  )

`summarise()` has grouped output by 'type'. You can override using the
`.groups` argument.


In [72]:
expression_filter$perc_expression = expression_filter$perc_expression / length(unique(data_long$sample_id))

In [73]:
nrow(expression_filter)

[1] 9151

In [74]:
nrow(unique(data_long[,c('type', 'variable')]))

[1] 9151

In [75]:
unique(expression_filter$type[expression_filter$perc_expression <= 0.2])

[1] "cytokine"

In [76]:
filtered_out = expression_filter[expression_filter$perc_expression <= 0.2,]

In [77]:
expression_filter = expression_filter[expression_filter$perc_expression > 0.2,]

In [78]:
#filtered_out

In [79]:
head(expression_filter,2)

type,variable,perc_expression
<chr>,<chr>,<dbl>
clinical_data,CK,1
clinical_data,CK_MB,1


In [80]:
nrow(expression_filter)

[1] 9145

In [81]:
head(expression_filter)

type,variable,perc_expression
<chr>,<chr>,<dbl>
clinical_data,CK,1.0000000
clinical_data,CK_MB,1.0000000
clinical_data,CRP,1.0000000
clinical_data,Troponin,1.0000000
cytokine,BCA1__CXCL13,0.8630137
cytokine,CTACK__CCL27,0.8630137


In [82]:
data_long = merge(data_long, expression_filter[,c('type', 'variable')], by.x = c('type', 'variable'), by.y = c('type', 'variable'))   # filter the data

In [83]:
unique(data_long$type)

[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
[5] "single_cell"

In [84]:
nrow(data_long)

[1] 1064734

In [86]:
length(unique(data_long$variable))

[1] 9145

In [87]:
length(unique(data_long$sample_id))

[1] 146

### Normalization & wide format

In [88]:
### Standardize values

In [89]:
standardize

[1] FALSE

In [90]:
if(standardize == TRUE){
    data_long = merge(data_long, data_long %>% group_by(variable) %>% summarise(mean = mean(value, na.rm = TRUE), sd = sd(value, na.rm = TRUE)))
    
    data_long[data_long == 0] = NA
    
    data_long = data_long[(data_long$sd != 0) & (!is.na(data_long$sd)),]
    
    data_nas = data.frame(is.na(data_long))
    data_long$value = (data_long$value - data_long$mean)/data_long$sd
    
    #data_long = data.frame(data_long)
    data_long$mean = NULL
    data_long$sd = NULL
    data_long$value[data_nas$value] = NA
    }

In [91]:
unique(data_long$type)

[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
[5] "single_cell"

In [92]:
## Prepare wide format for correlations

In [93]:
data_long$ident = paste0(data_long$type, '_0_', data_long$variable)

In [94]:
nrow(unique(data_long[,c('sample_id', 'ident')]))

[1] 1064734

In [95]:
nrow(data_long)

[1] 1064734

In [96]:
data_long[(data_long$sample_id == 'k11') & (data_long$variable == 'CRP'),]

,type,variable,X,sample_id,value,config,expression,ident
,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>,<lgl>,<chr>
295,clinical_data,CRP,1065493,k11,0.1375035,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-,TRUE,clinical_data_0_CRP


In [97]:
### Transform to wide

In [98]:
final_data = dcast(data_long, sample_id ~ ident , value.var = "value") # ! with this merging there might be NA values for some samples on some data types

In [99]:
head(final_data,2)

,sample_id,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_Troponin,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin__CCL11,⋯,single_cell_0_Treg__VIM,single_cell_0_Treg__YBX1,single_cell_0_Treg__YPEL5,single_cell_0_Treg__YWHAB,single_cell_0_Treg__YWHAZ,single_cell_0_Treg__ZC3HAV1,single_cell_0_Treg__ZFAS1,single_cell_0_Treg__ZFP36,single_cell_0_Treg__ZFP36L2,single_cell_0_Treg__ZNF331
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,k1,5.459432,NA,0.4854268,0.01863417,5.696550,9.821662,5.045268,10.562366,7.42341,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,k10,NA,NA,0.2630344,NA,4.697663,9.701220,1.922198,7.724173,9.19842,⋯,2.356913,1.735349,1.695393,1.772633,1.975818,0.9466424,1.417687,1.670544,1.474395,1.695393


In [100]:
rownames(final_data) = final_data$sample_id

In [101]:
final_data$sample_id = NULL

In [103]:
ncol(final_data)

[1] 9145

In [104]:
nrow(final_data)

[1] 146

### Deal with NA - Set NA for 0 observation + remove samples with only NA

In [106]:
set_zero_na

[1] FALSE

In [107]:
if(set_zero_na == TRUE){
    final_data[final_data == 0] = NA
    }

In [108]:
### Remember NA's

In [109]:
data_nas = is.na(final_data)

In [110]:
head(data_nas,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_Troponin,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin__CCL11,cytokine_0_Eotaxin2__CCL24,⋯,single_cell_0_Treg__VIM,single_cell_0_Treg__YBX1,single_cell_0_Treg__YPEL5,single_cell_0_Treg__YWHAB,single_cell_0_Treg__YWHAZ,single_cell_0_Treg__ZC3HAV1,single_cell_0_Treg__ZFAS1,single_cell_0_Treg__ZFP36,single_cell_0_Treg__ZFP36L2,single_cell_0_Treg__ZNF331
k1,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
k10,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [111]:
rowSums(data_nas)

k1   k10   k11   k12   k13   k14   k15   k16   k17   k18   k19    k2   k20 
 8185     3     1     2     1     1   893     2     1   894     2   895     3 
  k21   k22   k23   k24   k25   k26   k27   k28   k29    k3   k30   k31   k32 
  893     0   893   893     0     0   893     2  1386     4     0   893     2 
  k33   k34    k4    k5    k6    k7    k8    k9  m1.1  m1.2  m1.3  m1.4 m10.1 
    1     2   893     0     1     1     0   896   892  9145  9145  9145  7694 
m10.2 m10.3 m10.4 m11.1 m11.2 m11.3 m11.4 m12.1 m12.2 m12.3 m12.4 m13.1 m13.2 
    1     0  9145  8586     0     1     2   893     0  9145  9145     0   892 
m13.3 m13.4 m14.1 m14.2 m14.3 m14.4 m15.1 m15.2 m15.3 m15.4 m16.1 m16.2 m16.3 
    0     1     0   892  7694  7695     0   892     0     0   892     0  9145 
m16.4 m17.1 m17.2 m17.3 m17.4 m18.1 m18.2 m18.3 m18.4 m19.1 m19.2 m19.3 m19.4 
 9145     0  9145  9145  9145     0   892     1  9145     0     0   893  9145 
 m2.1  m2.2  m2.3  m2.4 m20.1 m20.2 m20.3 m20.4 m21.1 m21.2 m21.3 m21.4 m22.1 
    1   892   892   893  8587  7694   892     1     0     0     2     2     0 
m22.2 m22.3 m22.4 m23.1 m23.2 m23.3 m23.4 m24.1 m24.2 m24.3 m24.4 m25.1 m25.2 
    0   893     0     1   490  1447     1  8184     0     2     3     2     1 
m25.3 m25.4 m26.1 m26.2 m26.3 m26.4 m27.1 m27.2 m27.3 m27.4 m28.1 m28.2 m28.3 
    2  9145   490     0   892   556     1     0     0  9145  9145     0     1 
m28.4  m3.1  m3.2  m3.3  m3.4  m4.1  m4.2  m4.3  m4.4  m5.1  m5.2  m5.3  m5.4 
    2  9145     0     0     0   490     1     0     2     0  8586     0   893 
 m6.1  m6.2  m6.3  m6.4  m7.1  m7.2  m7.3  m7.4  m8.1  m8.2  m8.3  m8.4  m9.1 
 1382   893     0     1   892     1     0   894     1     1     2     3   892 
 m9.2  m9.3  m9.4 
  893     0  9145

In [112]:
ncol(final_data)

[1] 9145

In [113]:
keep_samples = names(rowSums(data_nas))[rowSums(data_nas) != ncol(final_data)]

In [114]:
final_data = final_data[keep_samples,]

In [115]:
data_nas = data_nas[keep_samples,]

In [116]:
nrow(final_data)

[1] 128

### Apply feature wise quantile normalization

In [136]:
quantile_norm_feat

[1] TRUE

In [137]:
if(quantile_norm_feat == TRUE){
    final_data = apply(final_data, 2,stdnorm)
    final_data = data.frame(final_data)
    final_data[data_nas] = NA
    final_data$sample_id = rownames(final_data)
    data_long = melt(final_data)
    data_long$type = str_extract(data_long$variable, '.*_0_')
    data_long$type  = str_replace(data_long$type , '_0_', '')
    data_long$variable = str_replace(data_long$variable, '.*_0_', '')
    }

Using sample_id as id variables



In [139]:
length(unique(data_long$variable))

[1] 9145

# Remove certain variables

In [234]:
unique(data_long$variable[data_long$type == 'clinical_data'])

[1] "CK"       "CK_MB"    "CRP"      "Troponin"

In [235]:
data_long = data_long[data_long$variable != 'delta_ef_value',]

In [236]:
head(data_long,2)

,sample_id,variable,value,type
,<chr>,<chr>,<dbl>,<chr>
1,k1,CK,-2.397022,clinical_data
2,k10,CK,NA,clinical_data


In [237]:
unique(data_long$variable[data_long$type == 'clinical_data'])

[1] "CK"       "CK_MB"    "CRP"      "Troponin"

In [238]:
length(unique(data_long$variable))

[1] 9145

# Save Prepared Data

In [239]:
name_save

[1] "V_AZIMUTH_INTEGRATED"

In [240]:
write.csv(data_long, paste0(result_path, '/G-Analysis/Combined_Data_', name_save, '.csv'))

In [242]:
length(unique(data_long$variable))

[1] 9145